In [1]:
from pymanopt.manifolds import Grassmann, ComplexGrassmann, SpecialOrthogonalGroup
import numpy as np
from compute_centroid import *
from grass_DR import *
from PNG import *
from sklearn.decomposition import PCA
from PGA import *
import matplotlib.pyplot as plt

# Example 1: 15 points on Gr(2, 6) embedded in Gr(2, 10) 

## Generate data

In [2]:
m = 6
n = 10
p = 2
N = 15
sig = 0.1
gr_low = Grassmann(m, p, N)
gr = Grassmann(n, p, N)
gr_map = Grassmann(n, m)

X_low = gr_low.rand() # N x m x p
A = gr_map.rand() # n x m
#B = np.random.normal(0, 0.1, (n, p)) # n x p
B = np.zeros((n,p))
AAT = np.matmul(A, A.T) 
IAATB = np.matmul(np.eye(n) - AAT, B)
X = np.array([np.linalg.qr(np.matmul(A, X_low[i]) + IAATB)[0] for i in range(N)]) # N x n x p
X = gr.exp(X, sig * gr.randvec(X)) # perturb the emdedded X

## Compute the score using PNG and apply PCA to the scores

In [3]:
scores_PNG = PNG(X)

n_c = 5
pca = PCA(n_components = n_c)
pca.fit(scores_PNG)
print(f'The ratios of expressed variance first {n_c} PCs.')
print(pca.explained_variance_ratio_.round(2))
print(f'The cumulative ratios of expressed variance first {n_c} PCs.')
print(np.cumsum(pca.explained_variance_ratio_).round(2))

Gr(2, 10) -> Gr(2, 9)
Gr(2, 9) -> Gr(2, 8)
Gr(2, 8) -> Gr(2, 7)
Gr(2, 7) -> Gr(2, 6)
Gr(2, 6) -> Gr(2, 5)
Gr(2, 5) -> Gr(2, 4)
Gr(2, 4) -> Gr(2, 3)
Gr(2, 3) -> Gr(1, 3)
Gr(1, 3) -> Gr(1, 2)
Gr(1, 2) -> NGM
The ratios of expressed variance first 5 PCs.
[0.41 0.22 0.13 0.1  0.07]
The cumulative ratios of expressed variance first 5 PCs.
[0.41 0.62 0.76 0.86 0.93]


## PGA

In [4]:
gr = Grassmann(n, p)
n_c = 5
pga = PGA(X, n_c, gr)

print(f'The ratios of expressed variance first {n_c} PCs.')
print(pga.explained_variance_ratio_.round(2))
print(f'The cumulative ratios of expressed variance first {n_c} PCs.')
print(np.cumsum(pga.explained_variance_ratio_).round(2))

The ratios of expressed variance first 5 PCs.
[0.23 0.22 0.2  0.13 0.11]
The cumulative ratios of expressed variance first 5 PCs.
[0.23 0.45 0.64 0.77 0.88]


# Example 2: 15 points on Gr(2, $C^4$) embedded in Gr(2, $C^5$) 

## Generate data

In [2]:
m = 4
n = 5
p = 2
N = 15
sig = 0.1
cgr_low = ComplexGrassmann(m, p, N)
cgr = ComplexGrassmann(n, p, N)
cgr_map = ComplexGrassmann(n, m)

X_low = cgr_low.rand() # N x m x p
A = cgr_map.rand() # n x m
#B = np.random.normal(0, 0.1, (n, p)) # n x p
B = np.zeros((n,p))
AAT = np.matmul(A, A.conj().T) 
IAATB = np.matmul(np.eye(n) - AAT, B)
X = np.array([np.linalg.qr(np.matmul(A, X_low[i]) + IAATB)[0] for i in range(N)]) # N x n x p
X = cgr.exp(X, sig * cgr.randvec(X)) # perturb the emdedded X

In [3]:
scores = PNG(X)

Gr(2, 5) -> Gr(2, 4)
Gr(2, 4) -> Gr(2, 3)
Gr(2, 3) -> Gr(1, 3)
Gr(1, 3) -> Gr(1, 2)
Gr(1, 2) -> NGM


In [4]:
n_c = 5
pca = PCA(n_components = n_c)
pca.fit(np.hstack((scores.real, scores.imag)))
print(f'The ratios of expressed variance first {n_c} PCs.')
print(pca.explained_variance_ratio_.round(2))
print(f'The cumulative ratios of expressed variance first {n_c} PCs.')
print(np.cumsum(pca.explained_variance_ratio_).round(2))

The ratios of expressed variance first 5 PCs.
[0.35 0.19 0.15 0.12 0.08]
The cumulative ratios of expressed variance first 5 PCs.
[0.35 0.53 0.68 0.8  0.88]


## PGA

In [ ]:
# ??
gr = ComplexGrassmann(n, p)
n_c = 5
pga = PGA_complex(X, n_c, gr)

print(f'The ratios of expressed variance first {n_c} PCs.')
print(pga.explained_variance_ratio_.round(2))
print(f'The cumulative ratios of expressed variance first {n_c} PCs.')
print(np.cumsum(pga.explained_variance_ratio_).round(2))

# Example 3: Supervised Principal Nested Grassmann

## Generate Data

In [22]:
m = 2
n = 5
p = 1
N = 50
n_class = 2
sig = 0.1

gr_low = Grassmann(m, p, N)
y = np.random.randint(n_class, size = N)
M = np.array([[[1],[0]],[[0],[1]]])
X_low = np.zeros((N, m, p))
# translate the X such that different classes are separated
gr = Grassmann(m, p)
for i in range(N):
    X_low[i] = gr.exp(M[y[i]], 0.5 * gr.randvec(M[y[i]]))

gr_map = Grassmann(n, m)
A = gr_map.rand()
#B = np.random.normal(0, 0.1, (n, p)) # n_class x n x p
B = np.zeros((n, p))
AAT = np.matmul(A, A.T) 
IAATB = np.matmul(np.eye(n) - AAT, B)


gr = Grassmann(n, p, N)

X = np.array([np.linalg.qr(np.matmul(A, X_low[i]) + IAATB)[0] for i in range(N)]) # N x n x p
X = gr.exp(X, sig * gr.randvec(X)) # perturb the emdedded X

## Compute the score using SPNG

In [23]:
scores_PNG = SPNG(X, y)

Gr(1, 5) -> Gr(1, 4)
Gr(1, 4) -> Gr(1, 3)
Gr(1, 3) -> Gr(1, 2)
Gr(1, 2) -> NGM
